In [1]:
import pandas as pd
import numpy as np
import re
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AdamW
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import pickle
import ast
import statistics as s
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
import nltk
from transformers import pipeline
import copy
from nltk.corpus import stopwords

/home/nvanthof/.local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# load word2vec model
word2vec_model = Word2Vec.load("word2vec_finetuned.model")
# load BERT model
BERT_model = torch.load('BERT_finetuned.pt')
# load dataframe
df = pd.read_csv('preprocessed_df.csv')

tokenizer = BertTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
#BERT_model = BertForMaskedLM.from_pretrained("GroNLP/bert-base-dutch-cased")
#w2v_model.intersect_word2vec_format(r"combined-160.txt", binary=False, lockf=1.0)
# https://github.com/clips/dutchembeddings

# skiplist (words that should not be corrected: names)
with open("ocr_names.txt", "rb") as fp:   # Unpickling
    ocr_names = pickle.load(fp)

In [125]:
df.columns

Index(['Unnamed: 0', 'identifier', 'gt text', 'ocr text', 'CER', 'WER',
       'WER (order independent)', 'dictionary lookup gt',
       'dictionary lookup ocr', 'jaccard_coefficient',
       'levenshtein_dist_normalized', 'source', 'word count gt',
       'word count ocr', 'year', 'century', 'old index', 'gt text org',
       'ocr text org', 'set', 'gt sentences matched', 'ocr sentences matched',
       'CER matched sentences', 'WER matched sentences',
       'avg sentence length gt (fuzzy matched)_x',
       'avg sentence length ocr (fuzzy matched)_x',
       'max sentence length gt (fuzzy matched)_x',
       'max sentence length ocr (fuzzy matched)_x',
       'sentences gt (fuzzy matched)_x', 'sentences ocr (fuzzy matched)_x',
       'word count gt (fuzzy matched)_x', 'word count ocr (fuzzy matched)_x',
       'longest_streak', 'avg_longest_streaks', 'avg_total_missing_words',
       'avg_perc_missing_words ', 'gt sentences matchated',
       'avg sentence length gt (fuzzy matched)_

In [4]:
# set hyperparameters for validation
#topn_detection = 1000
#topn_correction = 1000
#window = 5

In [3]:
def list_merger(lists):
    new_list = []
    for elem in lists:
        new_list = new_list + elem
    return new_list

In [3]:
def check_numbers(token):
    numbers = any(char.isdigit() for char in token)
    return numbers

False


In [5]:
def correct_sorted(candidates, sim_or_probs, LD): # sorts first by LD, then by similarity/probability
    paired_sorted = sorted(zip(LD,sim_or_probs,candidates),key = lambda x: (x[0],x[1]), reverse=True)
    LD,sim_or_probs,candidates = zip(*paired_sorted)
    correction = candidates[0]
    return correction
    
def correct_calculated(candidates, sim_or_probs, LD): # calculates a score from LD and normalised similarity/probability
    inv_LD = 1 - LD
    sim_or_probs = np.array(sim_or_probs)
    sim_or_probs = np.interp(sim_or_probs, (sim_or_probs.min(), sim_or_probs.max()), (0, 1)).tolist()
    score = sim_or_probs / inv_LD
    zipped_pairs = zip(score.tolist(), candidates)
    sorted_by_score = [x for _, x in sorted(zipped_pairs, reverse=True)]
    correction = sorted_by_score[0]
    return correction

def remove_stopwords(candidates, cosine, LD):
    #nltk.download('stopwords')
    stop_words = set(stopwords.words('dutch'))
    candidates_nostopwords = []
    cosine_nostopwords = []
    LD_nostopwords = []
    for i in range(len(candidates)):
        if candidates[i] not in stop_words:
            candidates_nostopwords.append(candidates[i])
            cosine_nostopwords.append(cosine[i])
            LD_nostopwords.append(LD[i])
    LD_nostopwords = np.array(LD_nostopwords)
    return candidates_nostopwords, cosine_nostopwords, LD_nostopwords

In [1]:
#detection and correction validation word2vec
error_positions = [] # the position of a word when it is an error in predictions
non_error_positions = [] # the position of a word when it is not an error in predictions
right_token_positions = []

rights_correct_sorted_list = []
wrongs_correct_sorted_list = []
rights_correct_sorted_nosw_list = []
wrongs_correct_sorted_nosw_list = []
rights_correct_calculated_list = []
wrongs_correct_calculated_list = []

def detection_and_correction_word2vec(row, w2v_model, ocr_names, window=5, topn_detection=1000):
    if row['set'] != 'val':
        return np.nan
    else:
        print('start')
        identifier = row['identifier']
        OCR_text = row['aligned_OCR_sentences']
        GT_text = row['aligned_GT_sentences']
        OCR_text = ast.literal_eval(OCR_text)
        GT_text = ast.literal_eval(GT_text)
        OCR_text = list_merger(OCR_text)
        GT_text = list_merger(GT_text)
        window_range = list(range(0,window))
        window_range = np.array(window_range) - ((window - 1) / 2)
        
        # keep track of positions in candidates
        error_positions_doc = []
        non_error_positions_doc = []
        right_token_positions_doc = []
        
        # keep track of performance
        rights_correct_sorted = 0
        wrongs_correct_sorted = 0
        rights_correct_sorted_nosw = 0
        wrongs_correct_sorted_nosw = 0
        rights_correct_calculated = 0
        wrongs_correct_calculated = 0
        
        for i in range(len(OCR_text)):
            try:
                if (OCR_text[i] in ocr_names) or (check_numbers(OCR_text[i]) == True) or (len(OCR_text[i]) <= 2) or (GT_text[i] == 'REMOVED')::
                    continue
                error = True
                if OCR_text[i] == GT_text[i]:
                    error = False
                context = []
                for j in window_range:
                if (i+j >= 0) and (i+j < len(OCR_text)) and j != 0:
                    #context.append(OCR_text[i+int(j)])
                    if any(str.isdigit(c) for c in  OCR_text[i+int(j)]) == True:
                        context.append('%NUMBER%')
                    elif OCR_text[i+int(j)] in ocr_names:
                        context.append('%NNP%')
                    else:
                        context.append(OCR_text[i+int(j)])
                    GT_context.append(GT_text[i+int(j)])
                else:
                    pass
                candidates = []
                cosines = []
                # calculate positions detection task
                for prediction in w2v_model.predict_output_word(context, topn=topn_detection):
                    candidates.append(prediction[0])
                    cosines.append(prediction[1]) 
                # remove punctuation except for hyphen from candidates
                candidates = [re.sub(r'[^\w\d\s\-]+', '', x) for x in candidates]
                try:
                    position = candidates.index(OCR_text[i])
                except ValueError:
                    position = topn_detection
                if error == True:
                    # where the error is in the detection list
                    error_positions_doc.append(position)
                    # find where the right word is in the candidates list (same as detection list)
                    try: 
                        position_right_token = candidates.index(GT_text[i])
                    except ValueError: 
                        position_right_token = topn_detection
                    print('position_right_token:', position_right_token)
                    right_token_positions_doc.append(position_right_token)
                    # try two correction methods
                    # first calculate the normalized LDs:
                    LD = np.array([fuzz.ratio(OCR_text[i], word)/100 for word in candidates])
                    # try sorting method
                    correction = correct_sorted(candidates, cosines, LD)
                    if correction == GT_text[i]:
                        rights_correct_sorted += 1
                    elif correction != GT_text[i]:
                        wrongs_correct_sorted += 1
                    print('OCR:', OCR_text[i])
                    print('correction sorted:', correction)
                    # try again the sorting methods, but without stopwords
                    candidates_nostopwords, cosine_nostopwords, LD_nostopwords = remove_stopwords(candidates, cosines, LD)
                    correction = correct_sorted(candidates_nostopwords, cosine_nostopwords, LD_nostopwords)
                    if correction == GT_text[i]:
                        rights_correct_sorted_nosw += 1
                    elif correction != GT_text[i]:
                        wrongs_correct_sorted_nosw += 1
                    # try score calculation method
                    correction = correct_calculated(candidates, cosines, LD)
                    if correction == GT_text[i]:
                        rights_correct_calculated += 1
                    elif correction != GT_text[i]:
                        wrongs_correct_calculated += 1
                    print('OCR:', OCR_text[i])
                    print('correction score:', correction)
                elif error == False:
                    # where the non error is in the detection list
                    non_error_positions_doc.append(position)
                # calculate positions correction task
                if error == True:
                    try:
                        right_token_position = candidates.index(GT_text[i])
                        right_token_positions_doc.append(right_token_position)
                    except ValueError:
                        right_token_positions_doc.append(topn_detection)
                    right_token_positions.append(right_token_positions_doc)
            except:
                pass
            
            
    #return error_positions, non_error_positions
    error_positions.append(error_positions_doc)
    non_error_positions.append(non_error_positions_doc)
    right_token_positions.append(right_token_positions_doc)
    
    # add performance to the list
    rights_correct_sorted_list.append(rights_correct_sorted)
    wrongs_correct_sorted_list.append(wrongs_correct_sorted)
    rights_correct_sorted_nosw_list.append(rights_correct_sorted_nosw)
    wrongs_correct_sorted_nosw_list.append(wrongs_correct_sorted_nosw)
    rights_correct_calculated_list.append(rights_correct_calculated)
    wrongs_correct_calculated_list.append(wrongs_correct_calculated)
    
    return identifier, error_positions, non_error_positions, right_token_positions, rights_correct_sorted_list, wrongs_correct_sorted_list, rights_correct_sorted_nosw_list, wrongs_correct_sorted_nosw_list, rights_correct_calculated_list, wrongs_correct_calculated_list   
    
#for index, row in df.iterrows():
#    detection_word2vec(row)
# df.loc[70]
fake_test_list_GT_aligned = """Een koekenpan of kortweg pan is een platte pan met een lang handvat.
De pan ontleent zijn naam aan het feit dat in zo'n pan pannenkoeken worden gebakken. Ook ander voedsel, zoals vlees, wordt in een koekenpan gebraden"""
fake_test_list_OCR_aligned = """Een hoekenpan of kortweg pan is een platte pan met een hang handvat.
De pan ontleent zijn naam haan het feit dat in zo'n pan pannenkoeken horden gebakken. Ook ander voedsel, zoals vlees, word in een hoekenpan gebraden"""
fake_test_list_GT_aligned = fake_test_list_GT_aligned.split('.')
fake_test_list_OCR_aligned = fake_test_list_OCR_aligned.split('.')
fake_test_list_GT_aligned = [x.split(' ') for x in fake_test_list_GT_aligned]
fake_test_list_OCR_aligned = [x.split(' ') for x in fake_test_list_OCR_aligned]
d = {'identifier': ['111'], 'aligned_OCR_sentences': [str(fake_test_list_OCR_aligned)], 'aligned_GT_sentences': [str(fake_test_list_GT_aligned)], 'set': ['val']}
df_probeer = pd.DataFrame(data=d)

for index, row in df.loc[df['set'].isin(['val'])].iterrows():
    if (index == 70) or (index == 72):
        print('index:', index)
        identifier, w2v_error_positions, w2v_non_error_positions, w2v_right_token_positions, rights_correct_sorted_list, wrongs_correct_sorted_list, rights_correct_sorted_nosw_list, wrongs_correct_sorted_nosw_list, rights_correct_calculated_list, wrongs_correct_calculated_list = detection_and_correction_word2vec(row, word2vec_model, ocr_names)
    else:
        print('passed')



NameError: name 'pd' is not defined

In [ ]:
print(rights_correct_sorted_list)
print(wrongs_correct_sorted_list)
print(rights_correct_sorted_nosw_list)
print(wrongs_correct_sorted_nosw_list)
print(rights_correct_calculated_list)
print(wrongs_correct_calculated_list)

In [249]:
# analysis of performance on validation set word2vec
#d = {'error_positions in detection': error_positions, 'non_error_positions in detection': non_error_positions}
#validation_df_positions = pd.DataFrame(data = d)
d = {'rights_correct_sorted_list': rights_correct_sorted_list, 'wrongs_correct_sorted_list': wrongs_correct_sorted_list, \
    'rights_correct_sorted_nosw_list': rights_correct_sorted_nosw_list, 'wrongs_correct_sorted_nosw_list': wrongs_correct_sorted_nosw_list, \
    'rights_correct_calculated_list': rights_correct_calculated_list, 'wrongs_correct_calculated_list': wrongs_correct_calculated_list}
validation_df_correction_methods_word2vec = pd.DataFrame(data = d)


print(error_positions)
print(non_error_positions)
#print(error_positions)
error_positions = list_merger(error_positions)
non_error_positions = list_merger(non_error_positions)
right_token_positions = list_merger(right_token_positions)
print('errors:', s.mean(error_positions))
print('non errors:', s.mean(non_error_positions))
print('right token in prediction:', s.mean(right_token_positions))
print(validation_df_correction_methods_word2vec.head())

[1000, 1000, 1000, 1000, 1000, 1000]
[1000, 1000, 1000, 1000, 1000, 1000, 7, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 60, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
errors: 1000
non errors: 919.4583333333334
right token in prediction: 911.3333333333334
   rights_correct_sorted_list  wrongs_correct_sorted_list  \
0                           1                           5   

   rights_correct_sorted_nosw_list  wrongs_correct_sorted_nosw_list  \
0                                1                                5   

   rights_correct_calculated_list  wrongs_correct_calculated_list  
0                               0                               6  


In [197]:
df.columns

Index(['Unnamed: 0', 'identifier', 'gt text', 'ocr text', 'CER', 'WER',
       'WER (order independent)', 'dictionary lookup gt',
       'dictionary lookup ocr', 'jaccard_coefficient',
       'levenshtein_dist_normalized', 'source', 'word count gt',
       'word count ocr', 'year', 'century', 'old index', 'gt text org',
       'ocr text org', 'set', 'gt sentences matched', 'ocr sentences matched',
       'CER matched sentences', 'WER matched sentences',
       'avg sentence length gt (fuzzy matched)_x',
       'avg sentence length ocr (fuzzy matched)_x',
       'max sentence length gt (fuzzy matched)_x',
       'max sentence length ocr (fuzzy matched)_x',
       'sentences gt (fuzzy matched)_x', 'sentences ocr (fuzzy matched)_x',
       'word count gt (fuzzy matched)_x', 'word count ocr (fuzzy matched)_x',
       'longest_streak', 'avg_longest_streaks', 'avg_total_missing_words',
       'avg_perc_missing_words ', 'gt sentences matchated',
       'avg sentence length gt (fuzzy matched)_

In [250]:
#detection and correction validation BERTje
error_positions = [] # the position of a word when it is an error in predictions
non_error_positions = [] # the position of a word when it is not an error in predictions
right_token_positions = []

rights_correct_sorted_list = []
wrongs_correct_sorted_list = []
rights_correct_sorted_nosw_list = []
wrongs_correct_sorted_nosw_list = []
rights_correct_calculated_list = []
wrongs_correct_calculated_list = []

def detection_and_correction_BERTje(row, BERT_model, tokenizer, ocr_names, topn_detection=1000):
    if row['set'] != 'val':
        return np.nan
    else:
        identifier = row['identifier']
        print('Val')
        OCR_text = row['aligned_OCR_sentences']
        GT_text = row['aligned_GT_sentences']
        print('OCR:', OCR_text)
        print('GT:', GT_text)
        OCR_text = ast.literal_eval(OCR_text)
        GT_text = ast.literal_eval(GT_text)
        print(OCR_text)
        
        # keep track of positions in candidates
        error_positions_doc = []
        non_error_positions_doc = []
        right_token_positions_doc = []
        
        # keep track of performance
        rights_correct_sorted = 0
        wrongs_correct_sorted = 0
        rights_correct_sorted_nosw = 0
        wrongs_correct_sorted_nosw = 0
        rights_correct_calculated = 0
        wrongs_correct_calculated = 0
        
        print(OCR_text)
        
        for j in range(len(OCR_text)):
            for i in range(len(OCR_text[j])):
                if (OCR_text[j][i] in ocr_names) or check_numbers( == True) or (len(OCR_text[j][i]) <= 2)  or (GT_text[j][i] == 'REMOVED'):
                    continue
                error = True
                if OCR_text[j][i] == GT_text[j][i]:
                    error = False
                candidates = []
                probabilities = []
                # calculate positions detection task
                sentence = copy.deepcopy(OCR_text[j])
                for t in range(len(sentence)):
                    if any(str.isdigit(c) for c in context[t]) == True:
                        context[t] = '%NUMBER%'
                    elif context[t] in ocr_names:
                        context[t] = '%NNP%'
                sentence[i] = '[MASK]'
                sentence = ' '.join(sentence)
                print(sentence)
                pipe = pipeline('fill-mask', model=BERT_model, tokenizer = tokenizer, top_k=topn_detection, devide='cuda0')
                for res in pipe(sentence):
                    candidates.append(res['token_str'].replace(' ', ''))
                    probabilities.append(res['score'])
                # remove punctuation except for hyphen from candidates
                candidates = [re.sub(r'[^\w\d\s\-]+', '', x) for x in candidates]
                try:
                    position = candidates.index(OCR_text[j][i])
                except ValueError:
                    position = topn_detection
                if error == True:
                    # where the error is in the detection list
                    error_positions_doc.append(position)
                    # find where the right word is in the candidates list (same as detection list)
                    try: 
                        position_right_token = candidates.index(GT_text[j][i])
                    except ValueError: 
                        position_right_token = topn_detection
                    print('position_right_token:', position_right_token)
                    right_token_positions_doc.append(position_right_token)
                    # try two correction methods
                    # first calculate the normalized LDs:
                    LD = np.array([fuzz.ratio(OCR_text[j][i], word)/100 for word in candidates])
                    # try sorting method
                    correction = correct_sorted(candidates, probabilities, LD)
                    if correction == GT_text[j][i]:
                        rights_correct_sorted += 1
                    elif correction != GT_text[j][i]:
                        wrongs_correct_sorted += 1
                    print('OCR:', OCR_text[j][i])
                    print('correction sorted:', correction)
                    # try again the sorting methods, but without stopwords
                    candidates_nostopwords, probabilities_nostopwords, LD_nostopwords = remove_stopwords(candidates, probabilities, LD)
                    correction = correct_sorted(candidates_nostopwords, probabilities_nostopwords, LD_nostopwords)
                    if correction == GT_text[j][i]:
                        rights_correct_sorted_nosw += 1
                    elif correction != GT_text[j][i]:
                        wrongs_correct_sorted_nosw += 1
                    # try score calculation method
                    correction = correct_calculated(candidates, probabilities, LD)
                    if correction == GT_text[j][i]:
                        rights_correct_calculated += 1
                    elif correction != GT_text[j][i]:
                        wrongs_correct_calculated += 1
                    print('OCR:', OCR_text[j][i])
                    print('correction score:', correction)
                elif error == False:
                    # where the non error is in the detection list
                    non_error_positions_doc.append(position)
                # calculate positions correction task
                if error == True:
                    try:
                        right_token_position = candidates.index(GT_text[j][i])
                        right_token_positions_doc.append(right_token_position)
                    except ValueError:
                        right_token_positions_doc.append(topn_detection)
                    right_token_positions.append(right_token_positions_doc)
            
            
    #return error_positions, non_error_positions
    error_positions.append(error_positions_doc)
    non_error_positions.append(non_error_positions_doc)
    right_token_positions.append(right_token_positions_doc)
    
    # add performance to the list
    rights_correct_sorted_list.append(rights_correct_sorted)
    wrongs_correct_sorted_list.append(wrongs_correct_sorted)
    rights_correct_sorted_nosw_list.append(rights_correct_sorted_nosw)
    wrongs_correct_sorted_nosw_list.append(wrongs_correct_sorted_nosw)
    rights_correct_calculated_list.append(rights_correct_calculated)
    wrongs_correct_calculated_list.append(wrongs_correct_calculated)
    
    return identifier, error_positions, non_error_positions, right_token_positions, rights_correct_sorted_list, wrongs_correct_sorted_list, rights_correct_sorted_nosw_list, wrongs_correct_sorted_nosw_list, rights_correct_calculated_list, wrongs_correct_calculated_list   
    
#for index, row in df.iterrows():
#    detection_word2vec(row)
# df.loc[70]
fake_test_list_GT_aligned = """Een koekenpan of kortweg pan is een platte pan met een lang handvat.
De pan ontleent zijn naam aan het feit dat in zo'n pan pannenkoeken worden gebakken. Ook ander voedsel, zoals vlees, wordt in een koekenpan gebraden"""
fake_test_list_OCR_aligned = """Een hoekenpan of kortweg pan is een platte pan met een hang handvat.
De pan ontleent zijn naam haan het feit dat in zo'n pan pannenkoeken horden gebakken. Ook ander voedsel, zoals vlees, word in een hoekenpan gebraden"""
fake_test_list_GT_aligned = fake_test_list_GT_aligned.split('.')
fake_test_list_OCR_aligned = fake_test_list_OCR_aligned.split('.')
fake_test_list_GT_aligned = [x.split(' ') for x in fake_test_list_GT_aligned]
fake_test_list_OCR_aligned = [x.split(' ') for x in fake_test_list_OCR_aligned]
d = {'identifier': ['111'], 'aligned_OCR_sentences': [str(fake_test_list_OCR_aligned)], 'aligned_GT_sentences': [str(fake_test_list_GT_aligned)], 'set': ['val']}
df_probeer = pd.DataFrame(data=d)

for index, row in df.loc[df['set'].isin(['val'])].iterrows():
    if (index == 70) or (index == 72):
        print('index:', index)
        identifier, BERT_error_positions, BERT_non_error_positions, BERT_right_token_positions, rights_correct_sorted_list, wrongs_correct_sorted_list, rights_correct_sorted_nosw_list, wrongs_correct_sorted_nosw_list, rights_correct_calculated_list, wrongs_correct_calculated_list = detection_and_correction_BERTje(row, BERT_model, tokenizer, ocr_names)
    else:
        print('passed')






Val
OCR: [['Een', 'hoekenpan', 'of', 'kortweg', 'pan', 'is', 'een', 'platte', 'pan', 'met', 'een', 'hang', 'handvat'], ['\nDe', 'pan', 'ontleent', 'zijn', 'naam', 'haan', 'het', 'feit', 'dat', 'in', "zo'n", 'pan', 'pannenkoeken', 'horden', 'gebakken'], ['', 'Ook', 'ander', 'voedsel,', 'zoals', 'vlees,', 'word', 'in', 'een', 'hoekenpan', 'gebraden']]
GT: [['Een', 'koekenpan', 'of', 'kortweg', 'pan', 'is', 'een', 'platte', 'pan', 'met', 'een', 'lang', 'handvat'], ['\nDe', 'pan', 'ontleent', 'zijn', 'naam', 'aan', 'het', 'feit', 'dat', 'in', "zo'n", 'pan', 'pannenkoeken', 'worden', 'gebakken'], ['', 'Ook', 'ander', 'voedsel,', 'zoals', 'vlees,', 'wordt', 'in', 'een', 'koekenpan', 'gebraden']]
[['Een', 'hoekenpan', 'of', 'kortweg', 'pan', 'is', 'een', 'platte', 'pan', 'met', 'een', 'hang', 'handvat'], ['\nDe', 'pan', 'ontleent', 'zijn', 'naam', 'haan', 'het', 'feit', 'dat', 'in', "zo'n", 'pan', 'pannenkoeken', 'horden', 'gebakken'], ['', 'Ook', 'ander', 'voedsel,', 'zoals', 'vlees,', 'word

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nvanthof/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Een hoekenpan of kortweg [MASK] is een platte pan met een hang handvat
Een hoekenpan of kortweg pan is [MASK] platte pan met een hang handvat
Een hoekenpan of kortweg pan is een [MASK] pan met een hang handvat
Een hoekenpan of kortweg pan is een platte [MASK] met een hang handvat
Een hoekenpan of kortweg pan is een platte pan [MASK] een hang handvat
Een hoekenpan of kortweg pan is een platte pan met [MASK] hang handvat
Een hoekenpan of kortweg pan is een platte pan met een [MASK] handvat
position_right_token: 123
OCR: hang
correction sorted: handig
OCR: hang
correction score: houten
Een hoekenpan of kortweg pan is een platte pan met een hang [MASK]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nvanthof/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



De [MASK] ontleent zijn naam haan het feit dat in zo'n pan pannenkoeken horden gebakken

De pan [MASK] zijn naam haan het feit dat in zo'n pan pannenkoeken horden gebakken

De pan ontleent [MASK] naam haan het feit dat in zo'n pan pannenkoeken horden gebakken

De pan ontleent zijn [MASK] haan het feit dat in zo'n pan pannenkoeken horden gebakken

De pan ontleent zijn naam [MASK] het feit dat in zo'n pan pannenkoeken horden gebakken
position_right_token: 0
OCR: haan
correction sorted: aan
OCR: haan
correction score: aan

De pan ontleent zijn naam haan [MASK] feit dat in zo'n pan pannenkoeken horden gebakken


...

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nvanthof/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 [MASK] ander voedsel, zoals vlees, word in een hoekenpan gebraden
 Ook [MASK] voedsel, zoals vlees, word in een hoekenpan gebraden
 Ook ander voedsel, [MASK] vlees, word in een hoekenpan gebraden
 Ook ander voedsel, zoals vlees, [MASK] in een hoekenpan gebraden
position_right_token: 0
OCR: word
correction sorted: word
OCR: word
correction score: word
 Ook ander voedsel, zoals vlees, word in [MASK] hoekenpan gebraden


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nvanthof/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/nvanthof/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':


 Ook ander voedsel, zoals vlees, word in een [MASK] gebraden
position_right_token: 246
OCR: hoekenpan
correction sorted: koekenpan
OCR: hoekenpan
correction score: restaurant
 Ook ander voedsel, zoals vlees, word in een hoekenpan [MASK]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nvanthof/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [251]:
# analysis of performance on validation set BERTje
#d = {'error_positions in detection': error_positions, 'non_error_positions in detection': non_error_positions}
#validation_df_positions = pd.DataFrame(data = d)
d = {'rights_correct_sorted_list': rights_correct_sorted_list, 'wrongs_correct_sorted_list': wrongs_correct_sorted_list, \
    'rights_correct_sorted_nosw_list': rights_correct_sorted_nosw_list, 'wrongs_correct_sorted_nosw_list': wrongs_correct_sorted_nosw_list, \
    'rights_correct_calculated_list': rights_correct_calculated_list, 'wrongs_correct_calculated_list': wrongs_correct_calculated_list}
validation_df_correction_methods_BERTje = pd.DataFrame(data = d)

print(BERT_right_token_positions)
#print(error_positions)
BERT_error_positions = list_merger(BERT_error_positions)
BERT_non_error_positions = list_merger(nBERT_on_error_positions)
BERT_right_token_positions = list_merger(BERT_right_token_positions)
print('errors:', s.mean(BERT_error_positions))
print('non errors:', s.mean(BERT_non_error_positions))
print('right token in prediction:', s.mean(BERT_right_token_positions))
validation_df_correction_methods_BERTje.head()

[[156, 156, 123, 123, 0, 0, 0, 0, 0, 0, 246, 246], [156, 156, 123, 123, 0, 0, 0, 0, 0, 0, 246, 246], [156, 156, 123, 123, 0, 0, 0, 0, 0, 0, 246, 246], [156, 156, 123, 123, 0, 0, 0, 0, 0, 0, 246, 246], [156, 156, 123, 123, 0, 0, 0, 0, 0, 0, 246, 246], [156, 156, 123, 123, 0, 0, 0, 0, 0, 0, 246, 246], [156, 156, 123, 123, 0, 0, 0, 0, 0, 0, 246, 246]]
errors: 851.8333333333334
non errors: 180.66666666666666
right token in prediction: 87.5


,rights_correct_sorted_list,wrongs_correct_sorted_list,rights_correct_sorted_nosw_list,wrongs_correct_sorted_nosw_list,rights_correct_calculated_list,wrongs_correct_calculated_list
0,3,3,2,4,2,4


In [37]:
text = "Dit1 is2 een3 zin4 die5 lang6 genoeg7 is8"
text = text.split(' ')

window = 5
window_range = list(range(0,window))
print(window_range)
window_range = np.array(window_range) - ((window - 1) / 2)
print(window_range)

context = []

for i in range(len(text)):
    print(i)
    print(text[i])
    context = []
    for j in window_range:
        if (i+j >= 0) and (i+j < len(text)) and j != 0:
            context.append(text[i+int(j)])
        else:
            pass
    print('context:', context)
    



[0, 1, 2, 3, 4]
[-2. -1.  0.  1.  2.]
0
Dit1
context: ['is2', 'een3']
1
is2
context: ['Dit1', 'een3', 'zin4']
2
een3
context: ['Dit1', 'is2', 'zin4', 'die5']
3
zin4
context: ['is2', 'een3', 'die5', 'lang6']
4
die5
context: ['een3', 'zin4', 'lang6', 'genoeg7']
5
lang6
context: ['zin4', 'die5', 'genoeg7', 'is8']
6
genoeg7
context: ['die5', 'lang6', 'is8']
7
is8
context: ['lang6', 'genoeg7']


In [103]:
word2vec_model.predict_output_word(['Een', 'of', 'kortweg'], topn=10)

[('den', 7.57105e-07),
 ('ende', 7.248402e-07),
 ('de', 7.188712e-07),
 ('noch', 7.1387456e-07),
 ('is', 7.0737497e-07),
 ('hebben', 7.070108e-07),
 ('vyand', 7.065082e-07),
 ('eenige', 7.0626095e-07),
 ('met', 7.061281e-07),
 ('wt', 7.042607e-07)]

In [102]:
word2vec_model.wv.most_similar('papier')

[('karton', 0.8278681039810181),
 ('inkt', 0.8026106357574463),
 ('rubberdoek', 0.7930652499198914),
 ('krantenpapier', 0.792934775352478),
 ('perkament', 0.7911171913146973),
 ('vloeipapier', 0.7774406671524048),
 ('vel', 0.7763333320617676),
 ('handgeschept', 0.771336555480957),
 ('potlood', 0.760221540927887),
 ('bedrukken', 0.7578753232955933)]

In [252]:
pipe = pipeline('fill-mask', model=BERT_model, tokenizer = tokenizer, top_k=10)
for res in pipe('Ik vind jou een [MASK].'):
    print(res['sequence'])
    print(res)

Ik vind jou een meisje.
{'sequence': 'Ik vind jou een meisje.', 'score': 0.06847196072340012, 'token': 15499, 'token_str': 'm e i s j e'}
Ik vind jou een vrouw.
{'sequence': 'Ik vind jou een vrouw.', 'score': 0.03620399907231331, 'token': 22098, 'token_str': 'v r o u w'}
Ik vind jou een vriendin.
{'sequence': 'Ik vind jou een vriendin.', 'score': 0.03159778192639351, 'token': 22057, 'token_str': 'v r i e n d i n'}
Ik vind jou een schatje.
{'sequence': 'Ik vind jou een schatje.', 'score': 0.030144749209284782, 'token': 18609, 'token_str': 's c h a t j e'}
Ik vind jou een vriend.
{'sequence': 'Ik vind jou een vriend.', 'score': 0.02135034091770649, 'token': 22053, 'token_str': 'v r i e n d'}
Ik vind jou een klootzak.
{'sequence': 'Ik vind jou een klootzak.', 'score': 0.017202582210302353, 'token': 14376, 'token_str': 'k l o o t z a k'}
Ik vind jou een trut.
{'sequence': 'Ik vind jou een trut.', 'score': 0.015790071338415146, 'token': 20399, 'token_str': 't r u t'}
Ik vind jou een jongen.

In [243]:
old_sentence = "Die [MASK] hadde viele avontuere ghemaect."
new_sentence = "De [MASK] had vele avonturen gemaakt."
pipe = pipeline('fill-mask', model=BERT_model, tokenizer = tokenizer, top_k=5)
for res in pipe(new_sentence):
    print(res['token_str'].replace(' ', ''))
    print(res['score'])
    pipe = pipeline('fill-mask', model=BERT_model, tokenizer = tokenizer, top_k=5)
for res in pipe(old_sentence):
    print(res['token_str'].replace(' ', ''))
    print(res['score'])

tweeling
0.03529518470168114
jongen
0.021275663748383522
Kameleon
0.016648495569825172
[UNK]
0.01608702912926674
dwerg
0.010880783200263977
[UNK]
0.019976049661636353
q
0.011574698612093925
##Ô
0.011502194218337536
##ô
0.011489509604871273
x
0.011456236243247986
